# Test simulation matching

TODO

- DONE: rewrite Table with chi2 < 5 and N_{chi2 < 5} for AGNdT8 and FBconst separated
- rewrite Table with phi_rel -> delta phi_rel where it is the mean +- std of variation from MCMC
- rewrite Table with correct Einstein radii

- for MCMC variations use:
``` python
from mcmc_eval import load_mcmcpkl, get_mcmcpeaks, get_mcmcstats
rawmcmc_chains, mcmc_chains = load_mcmcpkl('SDSSJ0029-0055', '{:03d}'.format(20))
```

- make a new grid plot with chi2 AGN vs FBconst for each lens in a separate panel

## Imports

In [1]:
import sys
import os
import pprint
import cPickle as pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
gleam_root = "/home/phdenzel/gleam"
sys.path.append(gleam_root)
from gleam.utils.lensing import LensModel
from gleam.utils.lensing import DL, DLSDS, DLDSDLS
from gleam.utils.lensing import kappa_profile, find_einstein_radius
from gleam.utils.plotting import kappa_map_plot
from gleam.utils.encode import an_sort
import gleam.utils.colors as gcl
gcl.GLEAMcmaps.register_all()
from mcmc_matching import load_lo, load_lm
from mcmc_eval import read_mcmctxt
from match_eval import read_matchlog


## Load lensing data and models

In [2]:
ids = ['SDSSJ0029-0055', 'SDSSJ0737+3216', 'SDSSJ0753+3416', 'SDSSJ0956+5100',
       'SDSSJ1051+4439', 'SDSSJ1430+6104', 'SDSSJ1627-0053']
idx = 2
lens = ids[idx]
pixrad = 11
print(lens)
mdir = 'match_plots/{lens}/'.format(lens=lens)
sdir = os.path.join(mdir, 'sorted')

mcmcmdl_range, chi2_mcmc, mcmcangles = read_mcmctxt('mcmc/mcmceval_{}.txt'.format(lens))
chi2_mcmc = -chi2_mcmc
if idx in [2, 3, 5]:
    chi2_mcmc = 0.25*chi2_mcmc
if idx in [6]:
    chi2_mcmc = 0.5*chi2_mcmc
# chi2_mcmc = np.exp(-2*chi2_mcmc)

matchlog = '{}{}_matching.log'.format(mdir, lens)
matchmdl_range, chi2_match, matchangles = read_matchlog(matchlog)
print(chi2_match[0])
nmodels = np.arange(0, len(chi2_match), 1.) / len(chi2_match)
chi2_match = chi2_match - chi2_match.min() + chi2_match[0]
chi2_match = np.sort(chi2_match) * np.exp(2*nmodels)
# chi2_match = 2*chi2_match
# if idx in [4, 5]:
#     chi2_match = 2*chi2_match
# chi2_match = np.exp(2*chi2_match)
# sortidcs = np.argsort(chi2_match)
matchmdl_range = matchmdl_range#[sortidcs]
chi2_match = chi2_match#[sortidcs]
matchangles = matchangles#[sortidcs]

# print(mcmcmdl_range[:11])
# print(chi2_mcmc[:11])
print(matchmdl_range[:11])
print(chi2_match[:11])


SDSSJ0753+3416
2.7752
[144 219 375 209 142 108 505 572 106 145 235]
[2.7752     2.821457   2.8423178  2.92648286 2.94785517 2.96841952
 2.98123664 3.01312392 3.0606731  3.07268432 3.10507746]


In [3]:
files = !ls models/kappa/*/*.fits
files_ansort = an_sort(files, argsort=True)
all_models, skipped = LensModel(files, return_skipped=True, verbose=True)
files = [f for i, f in enumerate(files) if i not in skipped]

filename            	lensmodels.fits
filepath            	lensmodels.fits
N                   	1577
pixrad              	80
maprad              	4.0
pixel_size          	0.0496894409938
kappa               	1.64243158638e+11
minima              	[]
saddle_points       	[]
maxima              	[]
zl                  	None
zs                  	None


In [4]:
# load stellar masses
M_stel_AGNT8 = np.loadtxt('models/MH_AGNT8.txt')
M_stel_FBC = np.loadtxt('models/MH_FBC.txt')
M_stel = np.concatenate([M_stel_AGNT8, M_stel_FBC])
M_stel = np.delete(M_stel[np.argsort(files_ansort)], skipped)
print M_stel.shape


(1577,)


In [5]:
lm = load_lm(lens, update_pixrad=pixrad, verbose=False)
lm.rescale(zl_new=0.23, zs_new=0.8)  # reverse rescaling for comparison
print(lm.__v__)


{'minima': array([], dtype=float64), 'pixrad': 80, 'grid_size': 161, 'obj_index': 0, 'maprad': 4.0000000000000036, 'saddle_points': array([], dtype=float64), 'maxima': array([], dtype=float64)}
filename            	SDSSJ0753+3416_kappa_re1.31_zl0.14zs0.96.pkl
filepath            	models/SDSSJ0753+3416_kappa_re1.31_zl0.14zs0.96.pkl
N                   	629
pixrad              	80
maprad              	4.0
pixel_size          	0.0496894409938
kappa               	29064666657.1
minima              	[]
saddle_points       	[]
maxima              	[]
zl                  	0.23
zs                  	0.8


In [6]:
def bruteforce_find(models, index, pool, only_best=True, verbose=False):
    target_idcs = []
    if isinstance(index, int):
        index = [index]
    for midx in index:
        differences = np.sum(np.abs(all_models.data[:] - lm[midx][:]), axis=(1, 2))
        kinpool = np.argsort(differences)
        if verbose:
            print(differences[kinpool][0]),
        if only_best:
            kinpool = kinpool[0]
        target_idcs.append(kinpool)
    return np.array(target_idcs)


In [7]:
%%script false
bestof = 51
mdlidcs = np.concatenate((mcmcmdl_range[:bestof], matchmdl_range[:bestof]))
kinpool = bruteforce_find(lm, mdlidcs, all_models, only_best=False, verbose=True)
print(kinpool[:, 0])


In [8]:
%%script false
for i, mdl_idx in enumerate(mdlidcs[:5]):
    print("#"*30 + "   " + str(i) + "   " + "#"*30)
    kappa_map_plot(lm, contours=True, mdl_index=mdl_idx, levels=12, log=True)
    plt.colorbar()
    # plt.xlim(-2.5, 2.5)
    # plt.ylim(-2.5, 2.5)
    plt.show()

    kappa_map_plot(all_models, contours=True, mdl_index=kinpool[i][0], levels=12, log=True)
    plt.colorbar()
    # plt.xlim(-2.5, 2.5)
    # plt.ylim(-2.5, 2.5)
    plt.show()


In [9]:
%%script false

bestof = 51
mdlidcs = np.concatenate((mcmcmdl_range[:bestof], matchmdl_range[:bestof]))
kinpool = bruteforce_find(lm, mdlidcs, all_models, only_best=False, verbose=True)
print(kinpool[:, 0])

# write best of files
with open('models/matches/{}_best{}_{}.txt'.format(lens, bestof, bestof), 'wb') as f:
    f.write('# {}\n'.format('\t'.join(['{:40s}'.format('filename'), 'chi2', 'rotation'])))
    for i, ik in enumerate(kinpool):
        print(files[ik[0]])
        if i == 0:
            f.write('# Best matches (according to MCMC):\n')
        elif i == bestof:
            f.write('#\n# Best matches (according to chi2):\n')
        if i // bestof == 0:
            leastsqr = chi2_mcmc[i]
            angle = mcmcangles[i][0]
        elif i // bestof == 1:
            leastsqr = chi2_match[i-51]
            angle = matchangles[i-51]
        filestr = files[ik[0]].replace('models/kappa/', '')
        filestr = '{:40s}'.format(filestr)
        chi2str = '{:8.4f}'.format(leastsqr)
        anglestr = '{:6.2f}'.format(angle)
        f.write('\t'.join([filestr, chi2str, anglestr]))
        f.write('\n')
os.system('say "All models have been processed"')

In [10]:
kinpool = bruteforce_find(lm, mcmcmdl_range, all_models, only_best=False, verbose=False)


In [17]:
lm = load_lm(lens, update_pixrad=pixrad, verbose=False)
# print 1+lm.zl #/ 0.23
print DL(lm.zl, lm.zs) / DL(0.23, 0.8)
print DLSDS(lm.zl, lm.zs) / DLSDS(0.23, 0.8)
print DLDSDLS(lm.zl, lm.zs) / DLDSDLS(0.23, 0.8)
print
kfactor = DLDSDLS(lm.zl, lm.zs) / DLDSDLS(0.23, 0.8)
print kfactor**2

lm.pixel_size

{'minima': array([], dtype=float64), 'pixrad': 80, 'grid_size': 161, 'obj_index': 0, 'maprad': 4.0000000000000036, 'saddle_points': array([], dtype=float64), 'maxima': array([], dtype=float64)}
0.9242394936687284
0.8558560164632033
1.079900679425165

1.1661854774229332


0.049689440993788865

In [12]:
# write best-of file but with all models
print(len(files), len(M_stel), len(chi2_match))

clmns = ['lens', 'model', 'FBconst', 'AGNT8', 'M_stel', 'chi2', 'rotation', 'rE']
df = pd.DataFrame(columns=clmns)

for i, ik in enumerate(kinpool):
    filestr = files[ik[0]].replace('models/kappa/', '')
    model_name = os.path.basename(filestr).replace('initgrid.kappa.fits', '')
    fbc = int('FBconst' in filestr)
    agnt8 = int('AGNT8' in filestr)
    mstel = M_stel[ik[0]]
    leastsqr = chi2_match[i]
    rotation = mcmcangles[i][0]-360 if mcmcangles[i][0]>180 else mcmcangles[i][0]
    # all_models.rescale(0.23, 0.8, lm.zl, lm.zs)
    profile = kappa_profile(all_models, mdl_index=ik[0])
    rE = find_einstein_radius(*profile) #* (1+lm.zl)**2
    
    row = lens, model_name, fbc, agnt8, mstel, leastsqr, rotation, rE
    # print dict(zip(clmns, row))
    df = df.append(dict(zip(clmns, row)), ignore_index=True)
    # print(lens, model_name, fbc, agnt8, mstel, leastsqr, rotation)

# df.describe()
df.head(20)

(1577, 1577, 629)


,lens,model,FBconst,AGNT8,M_stel,chi2,rotation,rE
0,SDSSJ0753+3416,HH214S0A0B0G90,0,1,9.688229e+09,2.775200,-1.31,0.441697
1,SDSSJ0753+3416,HH60S1A0B0G90,0,1,1.045654e+10,2.821457,-1.14,0.355666
2,SDSSJ0753+3416,HH142S0A0B90G0,1,0,2.688430e+10,2.842318,0.99,0.536856
3,SDSSJ0753+3416,HH48S1A90B0G0,0,1,1.551205e+10,2.926483,0.96,0.531356
4,SDSSJ0753+3416,HH210S0A0B90G0,0,1,9.342045e+09,2.947855,-1.17,0.431555
5,SDSSJ0753+3416,HH17S1A0B0G90,0,1,1.343692e+10,2.968420,0.92,0.510819
6,SDSSJ0753+3416,HH267S0A0B90G0,1,0,1.999873e+10,2.981237,0.96,0.613823
7,SDSSJ0753+3416,HH60S1A0B0G90,1,0,2.202881e+10,3.013124,-1.19,0.610050
8,SDSSJ0753+3416,HH172S0A0B0G90,0,1,1.499007e+10,3.060673,1.11,0.426557
9,SDSSJ0753+3416,HH221S0A0B90G0,0,1,1.072561e+10,3.072684,1.22,0.539361


In [13]:
%%script false
savename = 'models/matches/{}_matching.csv'.format(lens)
df.to_csv(savename)
savename

In [14]:
fzl = []
fdl = []
fdlsds = []
fdldsdls = []

for i, lens in enumerate(ids):
    lm = load_lm(lens, update_pixrad=pixrad, verbose=False)
    fzl.append(lm.zl)
    fdl.append(DL(lm.zl, lm.zs))
    fdlsds.append(DLSDS(lm.zl, lm.zs))
    fdldsdls.append(DLDSDLS(lm.zl, lm.zs))

{'minima': array([], dtype=float64), 'pixrad': 80, 'grid_size': 161, 'obj_index': 0, 'maprad': 4.0000000000000036, 'saddle_points': array([], dtype=float64), 'maxima': array([], dtype=float64)}
{'minima': array([], dtype=float64), 'pixrad': 80, 'grid_size': 161, 'obj_index': 0, 'maprad': 4.0000000000000036, 'saddle_points': array([], dtype=float64), 'maxima': array([], dtype=float64)}
{'minima': array([], dtype=float64), 'pixrad': 80, 'grid_size': 161, 'obj_index': 0, 'maprad': 4.0000000000000036, 'saddle_points': array([], dtype=float64), 'maxima': array([], dtype=float64)}
{'minima': array([], dtype=float64), 'pixrad': 80, 'grid_size': 161, 'obj_index': 0, 'maprad': 4.0000000000000036, 'saddle_points': array([], dtype=float64), 'maxima': array([], dtype=float64)}
{'minima': array([], dtype=float64), 'pixrad': 80, 'grid_size': 161, 'obj_index': 0, 'maprad': 4.0000000000000036, 'saddle_points': array([], dtype=float64), 'maxima': array([], dtype=float64)}
{'minima': array([], dtype=flo

In [16]:
gls_res = [0.9657, 1.0252, 1.3137, 1.3956, 1.4956, 1.1528, 1.2667]
gls_res = [0.92, 1.12, 1.31, 1.64, 1.52, 1.16, 1.39]
message = \
    "\nSDSSJ0029-0055 \t r_E: {:4.2f}\n".format(gls_res[0]) \
    + "SDSSJ0737+3216 \t r_E: {:4.2f}\n".format(gls_res[1]) \
    + "SDSSJ0753+3416 \t r_E: {:4.2f}\n".format(gls_res[2]) \
    + "SDSSJ0956+5100 \t r_E: {:4.2f}\n".format(gls_res[3]) \
    + "SDSSJ1051+4439 \t r_E: {:4.2f}\n".format(gls_res[4]) \
    + "SDSSJ1430+6104 \t r_E: {:4.2f}\n".format(gls_res[5]) \
    + "SDSSJ1627-0053 \t r_E: {:4.2f}\n".format(gls_res[6])
print(message)

mtc1_res = [0.633385, 1.284851, 0.531356, 1.605826, 0.948637, 0.686144, 1.054198]
message2 = \
    "\nSDSSJ0029-0055 \t r_E: {:4.2f}\n".format(mtc1_res[0]) \
    + "SDSSJ0737+3216 \t r_E: {:4.2f}\n".format(mtc1_res[1]) \
    + "SDSSJ0753+3416 \t r_E: {:4.2f}\n".format(mtc1_res[2]) \
    + "SDSSJ0956+5100 \t r_E: {:4.2f}\n".format(mtc1_res[3]) \
    + "SDSSJ1051+4439 \t r_E: {:4.2f}\n".format(mtc1_res[4]) \
    + "SDSSJ1430+6104 \t r_E: {:4.2f}\n".format(mtc1_res[5]) \
    + "SDSSJ1627-0053 \t r_E: {:4.2f}\n".format(mtc1_res[6])
print(message2)

print [g/m for g, m in zip(gls_res, mtc1_res)]
# print [0.23/f for f in fzl]
print [(1+zl) for f, zl in zip(fdl, fzl)]
print [DLSDS(0.23, 0.8)/f * f2/DL(0.23, 0.8) for f, f2 in zip(fdlsds, fdl)]
# print [(DLDSDLS(0.23, 0.8)/f) for f in fdldsdls]


SDSSJ0029-0055 	 r_E: 0.92
SDSSJ0737+3216 	 r_E: 1.12
SDSSJ0753+3416 	 r_E: 1.31
SDSSJ0956+5100 	 r_E: 1.64
SDSSJ1051+4439 	 r_E: 1.52
SDSSJ1430+6104 	 r_E: 1.16
SDSSJ1627-0053 	 r_E: 1.39


SDSSJ0029-0055 	 r_E: 0.63
SDSSJ0737+3216 	 r_E: 1.28
SDSSJ0753+3416 	 r_E: 0.53
SDSSJ0956+5100 	 r_E: 1.61
SDSSJ1051+4439 	 r_E: 0.95
SDSSJ1430+6104 	 r_E: 0.69
SDSSJ1627-0053 	 r_E: 1.05

[1.4525130844588996, 0.8716964068207131, 2.465390435037903, 1.0212812596134326, 1.6022988772312277, 1.690607219475795, 1.3185378837751542]
[1.227, 1.3223, 1.1371, 1.2405, 1.1634, 1.1688, 1.2076]
[0.9332962279583815, 2.0866766283644806, 0.5336190807590114, 1.5067645902857523, 0.7640082815171316, 0.7384681470290313, 1.0799006794251649]
